In [9]:
# Copyright 2024 NASA
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Rice mapping in Bhutan with U-Net using high resolution satellite imagery

### This notebook shows an example of exporting the images as tiles for use in prediction. We will use Google Earth Engine (GEE) for this.

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/SERVIR/servir-aces/blob/main/notebooks/export_image_for_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/SERVIR/servir-aces/blob/main/notebooks/export_image_for_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
</table>
</br>
</br>
</br>

This notebook is also available in this github repo: https://github.com/SERVIR/servir-aces. Navigate to the `notebooks` folder.

## Setup environment

In [1]:
!pip install servir-aces

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 676.9/676.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 47.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 28.6 MB/s

In [2]:
!git clone https://github.com/SERVIR/servir-aces

Cloning into 'servir-aces'...
remote: Enumerating objects: 671, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 671 (delta 42), reused 69 (delta 38), pack-reused 572
Receiving objects: 100% (671/671), 2.33 MiB | 16.83 MiB/s, done.
Resolving deltas: 100% (419/419), done.


Now the repo is downloaded. We will create an environment file file to place point to our training data and customize parameters for the model. To do this, we make a copy of the `.env.example` file provided.

Under the hood, all the configuration provided via the environment file are parsed as a config object and can be accessed programatically.

Note current version does not expose all the model intracacies through the environment file but future version may include those depending on the need.

In [3]:
!cp servir-aces/.env.example servir-aces/config.env

## Setup config file variables

Okay, now we have the `config.env` file, we will use this to provide our environments and parameters.

Note there are several parameters that can be changed. Let's start by changing the BASEDIR as below.

```
BASEDIR = "/content/"
```

We are trying to export RGBN from Planetscope mosiac. Since we are using growing season and pre-growing season information. Thus, we have 8 optical bands, namely `red_before`, `green_before`, `blue_before`, `nir_before`, `red_during`, `green_during`, `blue_during`, and  `nir_during`. In adidition, you can use `USE_ELEVATION` and `USE_S1` config to include the topographic and radar information. Since currently we are not including these, so we won't be settting these config values.

Similarly, we will be tiling to 256x256 pixels, so let's also change that. In addition, if you want to keep buffer on the export images buffer for prediction purpose, you can use `KERNEL_BUFFER` to specify that. Half this will extend on the sides of each patch. You can specify the size as tupe (e.g. 72 x 72). If zero is used; it will not buffer. I will keep this to zero for now.

We will also make sure our `GCS_PROJECT` is setup correctly.

```
# For model training, USE_ELEVATION extends FEATURES with "elevation" & "slope"
# USE_S1 extends FEATURES with "vv_asc_before", "vh_asc_before", "vv_asc_during", "vh_asc_during",
# "vv_desc_before", "vh_desc_before", "vv_desc_during", "vh_desc_during"
# In case these are not useful and you have other bands in your training data, you can do set
# USE_ELEVATION and USE_S1 to False and update FEATURES to include needed bands
USE_ELEVATION = False
USE_S1 = False

PATCH_SHAPE = (256, 256)

KERNEL_BUFFER = 0

GCS_PROJECT = "servir-ee"
```

Let's also change the `SCALE`. `SCALE` is the scale of output; we've put 10 for this. Next, we and `GCS_BUCKET` is the Google Cloud Bucket you want this to be exported. You can use `GCS_IMAGE_DIR` to specify where inside the `GCS_BUCKET` you want to keep your prediction image. Similarly, you can use `GCS_IMAGE_PREFIX` to specify the file name prefix for the export image. In general, you can use them to construct the `file_name_prefix` (equivalent to fileNamePrefix in the `Export.image.toCloudStorage` function). Let's say I have a bucket with the name `dl-book`, and I want to store the exported image inside the sub-directory `chapter-1/images` with `image_2021` as my file name prefix, equivalent to `gs://dl-book/chapter-1/images/image_2021*.TFRecord`, then the config settings looks as below.

```
SCALE = 10

GCS_BUCKET = "dl-book"
# prediction image directory
GCS_IMAGE_DIR = "chapter-1/images"
# prediction image prefix
GCS_IMAGE_PREFIX = "image_2021"
```

## Update the config file programtically

Let's make a dictionary so we can change these config settings programatically.

In [11]:
BASEDIR = "/content/" # @param {type:"string"}

USE_ELEVATION = "False" # @param {type:"string"}
USE_S1 = "False" # @param {type:"string"}
PATCH_SHAPE = "(256, 256)" # @param {type:"string"}
KERNEL_BUFFER = "0" # @param {type:"string"}

GCS_PROJECT = "servir-ee" # @param {type:"string"}

SCALE = "10" # @param {type:"string"}

GCS_BUCKET = "dl-book" # @param {type:"string"}
# prediction image directory
GCS_IMAGE_DIR = "chapter-1/images" # @param {type:"string"}
# prediction image prefix
GCS_IMAGE_PREFIX = "image_2021" # @param {type:"string"}

In [12]:
config_settings = {
    "BASEDIR" : BASEDIR,
    "USE_ELEVATION": USE_ELEVATION,
    "USE_S1": USE_S1,
    "PATCH_SHAPE": PATCH_SHAPE,
    "KERNEL_BUFFER": KERNEL_BUFFER,
    "SCALE": SCALE,
    "GCS_PROJECT": GCS_PROJECT,
    "GCS_BUCKET": GCS_BUCKET,
    "GCS_IMAGE_DIR": GCS_IMAGE_DIR,
    "GCS_IMAGE_PREFIX": GCS_IMAGE_PREFIX,
}


In [13]:
import dotenv

config_file = "servir-aces/config.env"

for config_key in config_settings:
    dotenv.set_key(dotenv_path=config_file,
                   key_to_set=config_key,
                   value_to_set=config_settings[config_key]
                   )


## Load config file variables

In [14]:
from aces import Config, EEUtils
import ee

In [15]:
config = Config(config_file)

BASEDIR: /content
DATADIR: /content/data
using features: ['red_before', 'green_before', 'blue_before', 'nir_before', 'red_during', 'green_during', 'blue_during', 'nir_during']
using labels: ['class']


## Initialize the Earth Engine session

In [16]:
ee.Authenticate()

True

In [17]:
EEUtils.initialize_session(use_highvolume=True, project=config.GCS_PROJECT)

## Prep for data export

We are focusing on Paro district of Bhutan. Let's start by importing necessary. We will focus on the rice growing elevation of Paro, which is roughly ~1500 msl to ~2600 msl. Therefore, we will use the following `Geometry` for this chapter.

In [18]:
paro_geom = ee.Geometry.Polygon(
        [[[89.25910506388209, 27.58540960195346],
          [89.25910506388209, 27.159794800895543],
          [89.58182845255396, 27.159794800895543],
          [89.58182845255396, 27.58540960195346]]], None, False)


Next we will importing our images to export to. We have already prepared the pre-growing and growing composite for Paro. Let's use that.

In [19]:
composite_before = ee.Image("projects/servir-sco-assets/assets/Bhutan/ACES_2/Paro_Rice_Composite_2021/composite_before")
composite_during = ee.Image("projects/servir-sco-assets/assets/Bhutan/ACES_2/Paro_Rice_Composite_2021/composite_during")


Selected needed bands and rename the bands for clarification. Then add the composites together.

In [20]:
bands = ["red", "green", "blue", "nir"]

composite_before = composite_before.select(bands)
composite_during = composite_during.select(bands)

composite_before = composite_before.regexpRename("$(.*)", "_before")
composite_during = composite_during.regexpRename("$(.*)", "_during")
image = composite_before.addBands(composite_during).toFloat()


In [21]:
if config.USE_ELEVATION:
    elevation = ee.Image("projects/servir-sco-assets/assets/Bhutan/ACES_2/elevationParo")
    slope = ee.Image("projects/servir-sco-assets/assets/Bhutan/ACES_2/slopeParo")
    image = image.addBands(elevation).addBands(slope).toFloat()
    config.FEATURES.extend(["elevation", "slope"])


In [22]:
if config.USE_S1:
    sentinel1_asc_before_composite = ee.Image("projects/servir-sco-assets/assets/Bhutan/Sentinel1Ascending2021/s1AscBefore")
    sentinel1_asc_during_composite = ee.Image("projects/servir-sco-assets/assets/Bhutan/Sentinel1Ascending2021/s1AscDuring")
    sentinel1_desc_before_composite = ee.Image("projects/servir-sco-assets/assets/Bhutan/Sentinel1Descending2021/s1DescBefore")
    sentinel1_desc_during_composite = ee.Image("projects/servir-sco-assets/assets/Bhutan/Sentinel1Descending2021/s1DescDuring")

    image = image.addBands(sentinel1_asc_before_composite).addBands(sentinel1_asc_during_composite).addBands(sentinel1_desc_before_composite).addBands(sentinel1_desc_during_composite).toFloat()
    config.FEATURES.extend(["vv_asc_before", "vh_asc_before", "vv_asc_during", "vh_asc_during",
                            "vv_desc_before", "vh_desc_before", "vv_desc_during", "vh_desc_during"])

Finally select the `FEATURES` and print the band names in the image.

In [23]:
image = image.select(config.FEATURES)
print("image", image.bandNames().getInfo())

image ['red_before', 'green_before', 'blue_before', 'nir_before', 'red_during', 'green_during', 'blue_during', 'nir_during']


Now let's specify patch and file dimensions.


In [24]:
format_options = {
  "patchDimensions": [config.PATCH_SHAPE_SINGLE, config.PATCH_SHAPE_SINGLE],
  "maxFileSize": 104857600,
  "compressed": True
}


As mentioned before, if you have `KERNEL_BUFFER`, we will add it here.

In [25]:
if config.KERNEL_BUFFER:
    format_options["kernelSize"] = config.KERNEL_BUFFER

Finally, let's setup the export task options.

In [26]:
# Setup the task
image_export_options = {
    "description": "export_task_for_prediction",
    "file_name_prefix": f"{config.GCS_IMAGE_DIR}/{config.GCS_IMAGE_PREFIX}",
    "bucket": config.GCS_BUCKET,
    "scale": config.SCALE,
    "file_format": "TFRecord",
    "region": paro_geom,
    "format_options": format_options,
    "max_pixels": 1e13,
}

print("image_export_options", image_export_options)


image_export_options {'description': 'export_task_for_prediction', 'file_name_prefix': 'chapter-1/images/image_2021', 'bucket': 'dl-book', 'scale': 10, 'file_format': 'TFRecord', 'region': ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              89.25910506388209,
              27.58540960195346
            ],
            [
              89.25910506388209,
              27.159794800895543
            ],
            [
              89.58182845255396,
              27.159794800895543
            ],
            [
              89.58182845255396,
              27.58540960195346
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
}), 'format_options': {'patchDimensions': [256, 256], 'maxFileSize': 104857600, 'compressed': True}, 'max_pixels': 10000000000000.0}


## Start the export task

Let's start the export task. For this we are using `cloud` export. The `export_image` of `EEUtils` class makes it easier to export the images. You can specify `cloud` or `asset` for your `export_type`. Once this run successfully, you can check your `Tasks` tab to view the job.

In [27]:
EEUtils.export_image(image, export_type=["cloud"], start_training=True, **image_export_options)


Exporting training data to gs://dl-book/chapter-1/images/image_2021..
